# Position Calculation

In [2]:
from astroquery.simbad import Simbad
from astropy import units as u 
from astropy.coordinates import SkyCoord


customsimbad=Simbad()
customsimbad.add_votable_fields('plx')

stars=["Alpha Cen A", "Alpha Cen B", "Proxima Cen"]


result=customsimbad.query_objects(stars)

result




C:\Users\supar\AppData\Local\Temp\ipykernel_9404\3733245359.py:7: DeprecationWarning: 'plx' has been renamed 'plx_value'. You'll see it appearing with its new name in the output table
  customsimbad.add_votable_fields('plx')


main_id,ra,dec,coo_err_maj,coo_err_min,coo_err_angle,coo_wavelength,coo_bibcode,plx_value,user_specified_id,object_number_id
,deg,deg,mas,mas,deg,,,mas,,
object,float64,float64,float32,float32,int16,str1,object,float64,object,int64
* alf Cen A,219.90205833170774,-60.83399268831004,34.17567,28.45653,90,O,2007A&A...474..653V,742.12,Alpha Cen A,1
* alf Cen B,219.89609628987276,-60.83752756558407,180.2203,171.3926,90,O,2007A&A...474..653V,742.12,Alpha Cen B,2
NAME Proxima Centauri,217.42894222160578,-62.67949018907555,0.024,0.0344,90,O,2020yCat.1350....0G,768.0665,Proxima Cen,3


In [3]:
import numpy as np
#α centauri A


ra=219.90205833170774
dec=-60.83399268831004
plx_value=742.12


γ=np.pi/180

def result_crtsn(m,n,r):
    plxarc=r/1000
    d=(1/plxarc)*3.086e16  #in m

    x=d*(np.cos(γ*n))*np.cos(γ*m)
    y=d*(np.cos(γ*n))*np.sin(γ*m)
    z=d*np.sin(γ*m)

    return (x,y,z)





In [4]:
result_crtsn(ra,dec,plx_value)

(np.float64(-1.5546447523881598e+16),
 np.float64(-1.2999796772302504e+16),
 np.float64(-2.667491257980127e+16))

# Velocity calculation

# Wrong velocity ❌❌

In [5]:
from astroquery.simbad import Simbad
import numpy as np

# Query Simbad with correct field names
custom = Simbad()
custom.add_votable_fields('ra', 'dec', 'plx_value', 'pmra', 'pmdec', 'rvz_radvel')

stars = ["Alpha Cen A", "Alpha Cen B", "Proxima Cen"]
res = custom.query_objects(stars)

# Convert all column names to lowercase for safety
res.rename_columns(res.colnames, [c.lower() for c in res.colnames])

k = 4.74047  # km/s per (mas/yr * pc)

def compute_velocity_array(row):
    ra = np.deg2rad(row['ra'])
    dec = np.deg2rad(row['dec'])
    plx = row['plx_value']
    pmra = row['pmra'] if row['pmra'] is not None else 0.0
    pmdec = row['pmdec'] if row['pmdec'] is not None else 0.0
    rv = row['rvz_radvel'] if row['rvz_radvel'] is not None else 0.0

    if plx == 0 or plx is None:
        raise ValueError(f"Parallax missing for {row['main_id']}")

    # Distance in parsec
    d = 1 / (plx * 1e-3)

    # Unit vectors
    cosd = np.cos(dec)
    sind = np.sin(dec)
    cosa = np.cos(ra)
    sina = np.sin(ra)

    e_r = np.array([cosd * cosa, cosd * sina, sind])
    e_alpha = np.array([-sina, cosa, 0.0])
    e_delta = np.array([-sind * cosa, -sind * sina, cosd])

    # Tangential and radial velocity
    v_tan = k * d * (pmra * e_alpha + pmdec * e_delta)
    v_rad = rv * e_r
    v_total = v_tan + v_rad

    return v_total  # numpy array [vx, vy, vz]

# Compute velocities
velocity_arrays = []
for r in res:
    try:
        velocity_arrays.append(compute_velocity_array(r))
    except Exception as e:
        print(f"Skipping {r['main_id']}: {e}")

# Print result
for star, v in zip(stars, velocity_arrays):
    print(f"{star}: vx, vy, vz = {v}")


Alpha Cen A: vx, vy, vz = [-17097.19011557  16339.39359873   1487.85871545]
Alpha Cen B: vx, vy, vz = [-18236.37097783  14847.35897894   2519.13324745]
Proxima Cen: vx, vy, vz = [-17529.07859679  15976.36463706   2197.96655367]


# Actual velocity in m/s ✅✅ :

In [11]:
from astroquery.simbad import Simbad
import numpy as np

def get_velocity_arrays(stars=None):
    if stars is None:
        stars = ["Alpha Cen A", "Alpha Cen B", "Proxima Cen"]

    # Query Simbad
    custom = Simbad()
    custom.add_votable_fields('ra', 'dec', 'plx_value', 'pmra', 'pmdec', 'rvz_radvel')
    res = custom.query_objects(stars)
    res.rename_columns(res.colnames, [c.lower() for c in res.colnames])
    
    # Conversion constant: mas/yr * pc -> km/s
    k = 4.74047

    def compute_velocity_array(row):
        ra = np.deg2rad(row['ra'])
        dec = np.deg2rad(row['dec'])
        plx = row['plx_value']  # in mas
        pmra = (row['pmra'] if row['pmra'] is not None else 0.0) / 1000
        pmdec = (row['pmdec'] if row['pmdec'] is not None else 0.0) / 1000
        rv = row['rvz_radvel'] if row['rvz_radvel'] is not None else 0.0

        if plx == 0 or plx is None:
            raise ValueError(f"Parallax missing for {row['main_id']}")

        # Distance in parsec
        d = 1 / (plx * 1e-3)

        # Unit vectors for ICRS
        cosd, sind, cosa, sina = np.cos(dec), np.sin(dec), np.cos(ra), np.sin(ra)
        e_r = np.array([cosd*cosa, cosd*sina, sind])
        e_alpha = np.array([-sina, cosa, 0.0])
        e_delta = np.array([-sind*cosa, -sind*sina, cosd])

        # Velocities
        v_tan = k * d * (pmra*e_alpha + pmdec*e_delta)
        v_rad = rv * e_r
        return v_tan + v_rad

    # Compute velocities and store in a NumPy array
    velocity_arrays = []
    for r in res:
        try:
            velocity_arrays.append(compute_velocity_array(r))
        except Exception as e:
            print(f"Skipping {r['main_id']}: {e}")

    # Convert list of arrays into a single NumPy array
    return np.array(velocity_arrays)*1000  # shape: (N_stars, 3)


# A test to import data from Stardata.py ✅✅

In [ ]:
from Stardata import get_velocity_arrays as gva 
from Stardata import pcalc2 as pc2
μ=gva()
Ψ=pc2()


m0=1.98847e30 #1 solar-mass
m=np.array([1.1*m0,0.907*m0,0.122*m0]) 

vbc=(m[0]*μ[0]+m[1]*μ[1]+m[2]*μ[2])/(m[0]+m[1]+m[2]) #Initial vel of barycenter

posbc=(m[0]*Ψ[0]+m[1]*Ψ[1]+m[2]*Ψ[2])/(m[0]+m[1]+m[2]) #Initial position of barycenter






array([[ -759.71789549,  -374.29074642, -3490.09971363],
       [  840.05231987,   422.4690183 ,  3939.58119246],
       [  604.60845011,   233.93788088,  2179.58642163]])